我们使用最小二乘法，求解每个方程的系数，并通过逐渐增加阶数，为模型定阶

In [1]:
import pickle
import numpy as np
import pandas as pd

with open('data/subdata_diff1.pkl', 'rb') as f:
    subdata_diff1 = pickle.load(f)
    
# 模型阶数从1开始逐一增加
rows, cols = subdata_diff1.shape
aicList = []
lmList = []

for p in range(1,11):
    baseData = None
    for i in range(p,rows):
        tmp_list = list(subdata_diff1[i,:]) + list(subdata_diff1[i-p:i].flatten())
        if baseData is None:
            baseData = [tmp_list]
        else:
            baseData = np.r_[baseData, [tmp_list]]
    X = np.c_[[1]*baseData.shape[0],baseData[:,cols:]]
    Y = baseData[:,0:cols]
    coefMatrix = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T,X)),X.T),Y)
    aic = np.log(np.linalg.det(np.cov(Y - np.matmul(X,coefMatrix),rowvar=False))) + 2*(coefMatrix.shape[0]-1)**2*p/baseData.shape[0]
    aicList.append(aic)
    lmList.append(coefMatrix)
    

with open('data/aicList.pkl', 'wb') as f:
    pickle.dump(aicList, f)
    
with open('data/lmList.pkl', 'wb') as f:
    pickle.dump(lmList, f)
    
#对比查看阶数和AIC
pd.DataFrame({"P":range(1,11),"AIC":aicList})

,P,AIC
0,1,13.580156
1,2,13.312225
2,3,13.543633
3,4,14.266087
4,5,15.512437
5,6,17.539047
6,7,20.457337
7,8,24.385459
8,9,29.438091
9,10,35.785909
